In [31]:
from datetime import datetime
import pandas as pd

import Graph_algorithms.a_star as a_star
import Graph_algorithms.graph_loader as loader
import Graph_algorithms.dijkstra as dijkstra
import Utilities.formatter as formatter

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
graph_time = loader.load_weighted_graph(criterion='time')
graph_changes = loader.load_weighted_graph(criterion='change')

stop_coords = a_star.load_stop_coords()

In [32]:
start_time = datetime.strptime("08:03:00", "%H:%M:%S")
total_cost, path, run_time = dijkstra.dijkstra_min_time(graph_time, "Muchobór Wielki", "Mroźna", start_time)

df = formatter.format_schedule_df(path, criterion="time", start_time=start_time)  
print(df.to_string(index=False))

print("\n\n\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME"],
    "value": [total_cost, run_time]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                          start_stop arrival_time                            end_stop   cost
 107       08:05:00                     Muchobór Wielki     08:06:00       Stanisławowska (W.K. Formaty) 180.00
 107       08:06:00       Stanisławowska (W.K. Formaty)     08:07:00                             Trawowa  60.00
 107       08:07:00                             Trawowa     08:08:00                       Krzemieniecka  60.00
 107       08:08:00                       Krzemieniecka     08:09:00                             Końcowa  60.00
 107       08:09:00                             Końcowa     08:11:00                        Ostrowskiego 120.00
 107       08:11:00                        Ostrowskiego     08:12:00                                 FAT  60.00
   5       08:12:00                                 FAT     08:13:00                              Hutmen  60.00
   5       08:13:00                              Hutmen     08:14:00  Bzowa (Centrum Historii Zajezdnia)

In [33]:
start_time = datetime.strptime("08:03:00", "%H:%M:%S")
total_cost, path, run_time = a_star.a_star_min_time(graph_time, stop_coords, "Muchobór Wielki", "Mroźna", start_time, max_speed=15)

df = formatter.format_schedule_df(path, criterion="time", start_time=start_time)  
print(df.to_string(index=False))

print("\n\n\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME"],
    "value": [total_cost, run_time]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                          start_stop arrival_time                            end_stop   cost
 107       08:05:00                     Muchobór Wielki     08:06:00       Stanisławowska (W.K. Formaty) 180.00
 107       08:06:00       Stanisławowska (W.K. Formaty)     08:07:00                             Trawowa  60.00
 107       08:07:00                             Trawowa     08:08:00                       Krzemieniecka  60.00
 107       08:08:00                       Krzemieniecka     08:09:00                             Końcowa  60.00
 107       08:09:00                             Końcowa     08:11:00                        Ostrowskiego 120.00
 107       08:11:00                        Ostrowskiego     08:12:00                                 FAT  60.00
   5       08:12:00                                 FAT     08:13:00                              Hutmen  60.00
   5       08:13:00                              Hutmen     08:14:00  Bzowa (Centrum Historii Zajezdnia)

In [34]:
start_time = datetime.strptime("08:00:00", "%H:%M:%S")
total_cost, path, run_time = a_star.a_star_min_changes(graph_changes, "Muchobór Wielki", "Mroźna",  start_time)

df = formatter.format_schedule_df(path, criterion="change", start_time=start_time)  
print(df.to_string(index=False))

print("\n\n\n")

data = {
    "metric": ["TOTAL COST", "RUN TIME"],
    "value": [total_cost, run_time]
}
df_summary = pd.DataFrame(data)
print(df_summary.to_string(index=False))

line departure_time                            start_stop arrival_time                              end_stop  cost
 132       08:01:00                       Muchobór Wielki     08:02:00            Muchobór Wielki (Roślinna)     0
 132       08:02:00            Muchobór Wielki (Roślinna)     08:03:00                              Tyrmanda     0
 132       08:03:00                              Tyrmanda     08:04:00       MIŃSKA (Rondo Rotm. Pileckiego)     0
 132       08:04:00       MIŃSKA (Rondo Rotm. Pileckiego)     08:06:00                        Rogowska (P+R)     0
 132       08:06:00                        Rogowska (P+R)     08:08:00               Strzegomska (krzyżówka)     0
 132       08:08:00               Strzegomska (krzyżówka)     08:09:00                           Nowodworska     0
 132       08:09:00                           Nowodworska     08:10:00                       Strzegomska 148     0
 132       08:10:00                       Strzegomska 148     08:11:00          